In [1]:
import os
import glob
import mysql.connector
import pandas as pd
import numpy as np
from sql_queries import *
import tables

In [2]:
conn = mysql.connector.connect( host = 'localhost', user = 'root', password = '32r306b')
cur = conn.cursor()

In [4]:
cur.execute("USE sparkifydb;")

In [2]:
def get_files(filepath, format):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,f'*.{format}'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
file_path = get_files('M:\Projects\Data ETL with Posgres\millionsongsubset\MillionSongSubset', "h5")

In [11]:
file_path[1]

'M:\\Projects\\Data ETL with Posgres\\millionsongsubset\\MillionSongSubset\\A\\A\\A\\TRAAADZ128F9348C2E.h5'

# Processing .h5 files

#### Getting and inserting -"songs table"- data:

In [12]:
song_data = tables.File(file_path[1])
song_data.root

/ (RootGroup) 'H5 Song File'
  children := ['analysis' (Group), 'metadata' (Group), 'musicbrainz' (Group)]

In [23]:
# insert_song_data(file_path[0])

In [8]:
variable = []

# song_id
variable.append(str(song_data.root.metadata.songs.cols.song_id[0]).split("'")[1])


# title
variable.append(str(song_data.root.metadata.songs.cols.title[0]).split("'")[1])

# artist_id
variable.append(str(song_data.root.metadata.songs.cols.artist_id[0]).split("'")[1])

# year
variable.append(song_data.root.musicbrainz.songs.cols.year[0])

# duration
variable.append(song_data.root.analysis.songs.cols.duration[0])

variable[3] = variable[3].item()
variable[4] = variable[4].item()


In [11]:
song_table_insert

'\n\nINSERT INTO songs (song_id, title, artist_id, year, duration)\n VALUES (%s, %s, %s, %s, %s)\n'

In [12]:
cur.execute(song_table_insert, variable)
conn.commit()

### getting and inserting -"artist table"- data

In [13]:
#artist_id
artist_table_data = []
artist_table_data.append(str(song_data.root.metadata.songs.cols.artist_id[0]).split("'")[1])

# artist_name
artist_table_data.append(str(song_data.root.metadata.songs.cols.artist_name[0]).split("'")[1])

#artist_location
artist_table_data.append(str(song_data.root.metadata.songs.cols.artist_location[0]).split("'")[1])

artist_table_data.append(song_data.root.metadata.songs.cols.artist_latitude[0])
artist_table_data.append(song_data.root.metadata.songs.cols.artist_longitude[0])

# converting latiude and longitude from numpy.float64 to default python int 
artist_table_data[3] = artist_table_data[3].item()
artist_table_data[4] = artist_table_data[4].item() 

In [13]:
for val in artist_table_data:
    print(val, "\t", type(val))

ARMJAGH1187FB546F3 	 <class 'str'>
The Box Tops 	 <class 'str'>
Memphis, TN 	 <class 'str'>
35.14968 	 <class 'float'>
-90.04892 	 <class 'float'>


In [14]:
artist_table_data

['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan]

In [26]:
# replace nan (which can't be inserted in sql) with None type
[None if isnan(val) else val for val in artist_table_data]

['ARKRRTF1187B9984DA', 'Sonora Santanera', '', None, None]

In [20]:
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False

In [15]:
cur.execute(artist_table_insert, artist_table_data)
conn.commit()

# Now we need to work with .json files

### Getting and inserting time table

In [3]:
json_files = get_files("M:\Projects\Data ETL with Posgres\millionsongsubset\log_data", "json")

In [4]:
filepath = json_files[0]
filepath

'M:\\Projects\\Data ETL with Posgres\\millionsongsubset\\log_data\\2018\\11\\2018-11-01-events.json'

In [10]:
df = pd.read_json(filepath, lines = True)
df.head(100)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [8]:
df = df.query("page == 'NextSong'")
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1540266185796,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [13]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
t = pd.to_datetime(df['ts'], unit = 'ms')

In [15]:
time_list = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year,
                     t.dt.weekday))
columns_label = (('start_time' ,'hour', 'day', 'week', 'month', 'year', 'weekday'))

<ipython-input-15-4b919f58f259>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_list = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year,


In [23]:
time_list = list((t, df['ts'].dt.hour, df['ts'].dt.day, df['ts'].dt.weekofyear, df['ts'].dt.month, df['ts'].dt.year,
                     df['ts'].dt.weekday))
columns_label = (('start_time' ,'hour', 'day', 'week', 'month', 'year', 'weekday'))

<ipython-input-23-c6eaf790fc9b>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_list = list((t, df['ts'].dt.hour, df['ts'].dt.day, df['ts'].dt.weekofyear, df['ts'].dt.month, df['ts'].dt.year,


In [24]:
time_df = pd.DataFrame.from_dict(dict(zip(columns_label, time_list)))
time_df

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3
8,2018-11-01 21:24:53.796,21,1,44,11,2018,3
9,2018-11-01 21:28:54.796,21,1,44,11,2018,3
10,2018-11-01 21:42:00.796,21,1,44,11,2018,3
12,2018-11-01 21:52:05.796,21,1,44,11,2018,3
13,2018-11-01 21:55:25.796,21,1,44,11,2018,3


In [25]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

### User table

In [9]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


In [12]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

### Songplay Table

In [1]:
for index, row in df.iterrows():
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None
    
    # inserting songplay record
    songplay_record = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId,\
                       row.location, row.userAgent)
    
    cur.execute(songplay_table_insert, songplay_record)
    conn.commit()